## Setup

In [1]:
# Import system modules
import arcpy
from arcpy import env
from arcpy.sa import *
# Set Workspace environment
env.workspace = "C:/Users/mbindl/Documents/SCHOOL/Geog777/Project1/Geog777_Project_1/Geog777_Project_1.gdb"
# Set Overwrite option
env.OverwriteOutput = True

In [2]:
arcpy.ListFeatureClasses()

['Well_Nitrate',
 'CancerRate_County',
 'CancerRate_CensusTract',
 'hexbin',
 'OLS']

## Inverse Distance Weighted 

In [3]:
# Interpolate a series of nitrate well values 
#  saved as point features onto a rectangular 
#   raster using Inverse Distance Weighting (IDW).
# Requirements: Spatial Analyst Extension

# Set local variables
outRaster = "idw_nitrate"
inPointFeatures = "Well_Nitrate"
zField = "nitr_con"
cellSize = 0.01
k = input("Enter Number:")
searchRadius = RadiusVariable('', 12)

# delete output if it exists
if arcpy.Exists(outRaster):
    arcpy.Delete_management(outRaster)
    
# Execute IDW
outIDW = Idw(inPointFeatures, zField, cellSize, k, searchRadius)

# Save the output 
outIDW.save(outRaster)

Enter Number:2


## Generate Tesselation

In [4]:
# Generate Tesselation

# delete feature class if it already exists
if arcpy.Exists("hexbin"):
    arcpy.Delete_management("hexbin")

tessellation_extent = arcpy.Extent(-10340405, 5234953, -9656979, 5992793)
spatial_ref = arcpy.SpatialReference(3857)
arcpy.GenerateTessellation_management("hexbin",
                                      tessellation_extent, "HEXAGON",
                                      "500 SquareMiles", spatial_ref)

# Set local variables
inFeatures = "hexbin"
fieldName1 = "ID"
fieldName2 = "CancerRate"
fieldName3 = "NitrateRate"
fieldName4 = "Residual"
fieldName5 = "Estimated"
fieldName6 = "StdResidual"

# Execute AddField twice for two new fields
arcpy.AddField_management(inFeatures, fieldName1, "SHORT")
arcpy.AddField_management(inFeatures, fieldName2, "DOUBLE")
arcpy.AddField_management(inFeatures, fieldName3, "DOUBLE")
arcpy.AddField_management(inFeatures, fieldName4, "DOUBLE")
arcpy.AddField_management(inFeatures, fieldName5, "DOUBLE")
arcpy.AddField_management(inFeatures, fieldName6, "DOUBLE")

# create unqique integer id field for OLS to use
updateFieldsList = ['OBJECTID', 'ID']
with arcpy.da.UpdateCursor(inFeatures, updateFieldsList) as cursor:
    for row in cursor:
        row[1] = row[0]
        cursor.updateRow(row)

AttributeError: 'ToolValidator' object has no attribute 'isLicensed'

AttributeError: 'ToolValidator' object has no attribute 'isLicensed'

## Convert Tract or County to Raster

In [6]:
# Delete output raster if it exists
if arcpy.Exists("cancer_rate"):
    arcpy.Delete_management("cancer_rate")
# Set local variables
inFeature = "CancerRate_CensusTract"
outRaster = "cancer_rate"
# cellSize = 0.01
field = "canrate"

# Execute FeatureToRaster
arcpy.FeatureToRaster_conversion(inFeature, field, outRaster)

<Result 'C:/Users/mbindl/Documents/SCHOOL/Geog777/Project1/Geog777_Project_1/Geog777_Project_1.gdb\\cancer_rate'>

## Zonal Statistics as a Table to Aggregate Nitrate IDW to Hex Bin

In [7]:
# ZonalStatisticsAsTable
# Set local variables
inZoneData = "hexbin"
zoneField = "GRID_ID"
inValueRaster = "idw_nitrate"
outTable = "zonalstat_nitrate"

if arcpy.Exists(outTable):
    arcpy.Delete_management(outTable)
    
# Execute ZonalStatisticsAsTable
outZSaT = ZonalStatisticsAsTable(inZoneData, zoneField, inValueRaster, 
                                 outTable, "NODATA", "MEAN")

In [8]:
# ZonalStatisticsAsTable
# Set local variables
inZoneData = "hexbin"
zoneField = "GRID_ID"
inValueRaster = "cancer_rate"
outTable = "zonalstat_tract_cancer"

if arcpy.Exists("zonalstat_tract_cancer"):
    arcpy.Delete_management("zonalstat_tract_cancer")
    
# Execute ZonalStatisticsAsTable
outZSaT = ZonalStatisticsAsTable(inZoneData, zoneField, inValueRaster, 
                                 outTable, "NODATA", "MEAN")

## Join Table to Tesselation

In [9]:
def fieldJoinCalc(updateFC, updateFieldsList, sourceFC, sourceFieldsList):
    from time import strftime  
    print ("Started data transfer: " + strftime("%Y-%m-%d %H:%M:%S"))
   
    # Use list comprehension to build a dictionary from a da SearchCursor  
    valueDict = {r[0]:(r[1:]) for r in arcpy.da.SearchCursor(sourceFC, sourceFieldsList)}  
   
    with arcpy.da.UpdateCursor(updateFC, updateFieldsList) as updateRows:  
        for updateRow in updateRows:  
            # store the Join value of the row being updated in a keyValue variable  
            keyValue = updateRow[0]  
            # verify that the keyValue is in the Dictionary  
            if keyValue in valueDict:  
                # transfer the value stored under the keyValue from the dictionary to the updated field.  
                updateRow[1] = valueDict[keyValue][0]  
                updateRows.updateRow(updateRow)    
    del valueDict  
    print ("Finished data transfer: " + strftime("%Y-%m-%d %H:%M:%S"))

## Transfer Zonal Stat Table Values to Hexbins

In [10]:
# transfer attributes to tesselation layer
fieldJoinCalc('hexbin', ['GRID_ID', 'NitrateRate'], 'zonalstat_nitrate', ['GRID_ID', 'MEAN'])
print ("The 'Nitrate Rate' field in the hexbin data has been updated")

Started data transfer: 2020-02-29 23:41:02
Finished data transfer: 2020-02-29 23:41:02
The 'Nitrate Rate' field in the hexbin data has been updated


In [11]:
# transfer attributes to tesselation layer
fieldJoinCalc('hexbin', ['GRID_ID', 'CancerRate'], 'zonalstat_tract_cancer', ['GRID_ID', 'MEAN'])
print ("The 'Cancer Rate' field in the hexbin data has been updated")

Started data transfer: 2020-02-29 23:41:03
Finished data transfer: 2020-02-29 23:41:03
The 'Cancer Rate' field in the hexbin data has been updated


## Ordinary Least Squares

In [12]:
olsCoefTab = 'olsCoefTab'
olsDiagTab = 'olsDiagTab'
ols = "OLS"
try:
    if arcpy.Exists(ols):
        arcpy.Delete_management(ols)
    if arcpy.Exists(olsCoefTab):
        arcpy.Delete_management(olsCoefTab)
    if arcpy.Exists(olsDiagTab):
        arcpy.Delete_management(olsDiagTab)
    
    arcpy.OrdinaryLeastSquares_stats("hexbin", "ID",ols, 
                                 "CancerRate","NitrateRate",
                                 olsCoefTab, olsDiagTab)

    # transfer attributes to hexbin
    fieldJoinCalc('hexbin', ['ID', 'Residual'], 'OLS', ['ID', 'Residual'])
    print("The 'Residual' field in the hexbin data has been updated")

    fieldJoinCalc('hexbin', ['ID', 'Estimated'], 'OLS', ['ID', 'Estimated'])
    print("The 'Estimated' field in the hexbin data has been updated")

    fieldJoinCalc('hexbin', ['ID', 'StdResidual'], 'OLS', ['ID', 'StdResid'])
    print ("The 'StdResidual' field in the hexbin data has been updated")

except:
    # If an error occurred when running the tool, print out the error message.
    print(arcpy.GetMessages())

Started data transfer: 2020-02-29 23:41:17
Finished data transfer: 2020-02-29 23:41:17
The 'Residual' field in the hexbin data has been updated
Started data transfer: 2020-02-29 23:41:17
Finished data transfer: 2020-02-29 23:41:17
The 'Estimated' field in the hexbin data has been updated
Started data transfer: 2020-02-29 23:41:17
Finished data transfer: 2020-02-29 23:41:17
The 'StdResidual' field in the hexbin data has been updated


In [13]:
# transfer attributes to residual values to Layer
fieldJoinCalc('hexbin', ['ID', 'Residual'], 'OLS', ['ID', 'Residual'])
print ("The 'Residual' field in the hexbin data has been updated")

Started data transfer: 2020-02-29 23:41:17
Finished data transfer: 2020-02-29 23:41:17
The 'Residual' field in the hexbin data has been updated


In [14]:
# transfer attributes to hexbin Layer
fieldJoinCalc('hexbin', ['ID', 'Estimated'], 'OLS', ['ID', 'Estimated'])
print ("The 'Estimated' field in the hexbin data has been updated")

Started data transfer: 2020-02-29 23:41:17
Finished data transfer: 2020-02-29 23:41:17
The 'Estimated' field in the hexbin data has been updated


In [15]:
fieldJoinCalc('hexbin', ['ID', 'StdResidual'], 'OLS', ['ID', 'StdResid'])
print ("The 'StdResidual' field in the hexbin data has been updated")

Started data transfer: 2020-02-29 23:41:17
Finished data transfer: 2020-02-29 23:41:17
The 'StdResidual' field in the hexbin data has been updated


## Moran's I

In [16]:
arcpy.SpatialAutocorrelation_stats("Well_Nitrate", "nitr_con","NO_REPORT")

<Result '0.752633'>

In [17]:
arcpy.SpatialAutocorrelation_stats("OLS", "Residual","NO_REPORT")

<Result '0.355654'>

In [18]:
arcpy.SpatialAutocorrelation_stats("CancerRate_County", "canrate","NO_REPORT")

<Result '-0.248819'>

In [19]:
arcpy.SpatialAutocorrelation_stats("CancerRate_CensusTract", "canrate","NO_REPORT")

<Result '0.218969'>

In [ ]:
# Analyze the growth of regional per capita incomes in US
# Counties from 1969 -- 2002 using Ordinary Least Squares Regression

# Import system modules
import arcpy

# Set property to overwrite existing outputs
arcpy.env.overwriteOutput = True

# Local variables...
workspace = r"C:\Data"

try:
    if arcpy.Exists(ols):
        arcpy.Delete_management(ols)
    if arcpy.Exists(olsCoefTab):
        arcpy.Delete_management(olsCoefTab)
    if arcpy.Exists(olsDiagTab):
        arcpy.Delete_management(olsDiagTab)
    
    arcpy.OrdinaryLeastSquares_stats("hexbin", "ID",ols, 
                                 "CancerRate","NitrateRate",
                                 olsCoefTab, olsDiagTab)

    # Create Spatial Weights Matrix (Can be based off input or output FC)
    # Process: Generate Spatial Weights Matrix... 
    swm = arcpy.GenerateSpatialWeightsMatrix_stats("USCounties.shp", "MYID",
                        "euclidean6Neighs.swm",
                        "K_NEAREST_NEIGHBORS",
                        "#", "#", "#", 6) 
                        

    # Calculate Moran's I Index of Spatial Autocorrelation for 
    # OLS Residuals using a SWM File.  
    # Process: Spatial Autocorrelation (Morans I)...      
    moransI = arcpy.SpatialAutocorrelation_stats("OLS", "Residual",
                        "NO_REPORT", "GET_SPATIAL_WEIGHTS_FROM_FILE", 
                        "EUCLIDEAN_DISTANCE", "NONE", "#", 
                        "euclidean6Neighs.swm")

except:
    # If an error occurred when running the tool, print out the error message.
    print(arcpy.GetMessages())

## Inspect Database

In [105]:
arcpy.ListDatasets()

['idw_nitrate', 'cancer_rate']

In [106]:
arcpy.ListFeatureClasses()

['Well_Nitrate',
 'CancerRate_County',
 'CancerRate_CensusTract',
 'OLS',
 'hexbin']

In [120]:
arcpy.ListTables()

['zonalstat_nitrate', 'zonalstat_tract_cancer', 'olsCoefTab', 'olsDiagTab']

In [121]:
fields = arcpy.ListFields("hexbin")

for field in fields:
    print("{0} is a type of {1} with a length of {2}"
          .format(field.name, field.type, field.length))

OBJECTID is a type of OID with a length of 4
Shape is a type of Geometry with a length of 0
Shape_Length is a type of Double with a length of 8
Shape_Area is a type of Double with a length of 8
GRID_ID is a type of String with a length of 12
ID is a type of SmallInteger with a length of 2
CancerRate is a type of Double with a length of 8
NitrateRate is a type of Double with a length of 8
Residual is a type of Double with a length of 8
Estimated is a type of Double with a length of 8


In [122]:
fields = arcpy.ListFields("OLS")

for field in fields:
    print("{0} is a type of {1} with a length of {2}"
          .format(field.name, field.type, field.length))

OBJECTID is a type of OID with a length of 4
Shape is a type of Geometry with a length of 0
ID is a type of SmallInteger with a length of 2
CancerRate is a type of Double with a length of 8
NitrateRate is a type of Double with a length of 8
Shape_Length is a type of Double with a length of 8
Shape_Area is a type of Double with a length of 8
Estimated is a type of Double with a length of 8
Residual is a type of Double with a length of 8
StdResid is a type of Double with a length of 8


In [109]:
fields = arcpy.ListFields("zonalstat_nitrate")

for field in fields:
    print("{0} is a type of {1} with a length of {2}"
          .format(field.name, field.type, field.length))

OBJECTID is a type of OID with a length of 4
GRID_ID is a type of String with a length of 4
ZONE_CODE is a type of Integer with a length of 4
COUNT is a type of Integer with a length of 4
AREA is a type of Double with a length of 8
MEAN is a type of Double with a length of 8


In [110]:
fields = arcpy.ListFields("zonalstat_tract_cancer")

for field in fields:
    print("{0} is a type of {1} with a length of {2}"
          .format(field.name, field.type, field.length))

OBJECTID is a type of OID with a length of 4
GRID_ID is a type of String with a length of 4
ZONE_CODE is a type of Integer with a length of 4
COUNT is a type of Integer with a length of 4
AREA is a type of Double with a length of 8
MEAN is a type of Double with a length of 8


## References
* https://pro.arcgis.com/en/pro-app/tool-reference/spatial-analyst/idw.htm
* https://pro.arcgis.com/en/pro-app/tool-reference/data-management/generatetesellation.htm
* https://pro.arcgis.com/en/pro-app/tool-reference/spatial-analyst/zonal-statistics-as-table.htm
* https://pro.arcgis.com/en/pro-app/tool-reference/spatial-statistics/ordinary-least-squares.htm